<table style="margin: 0; text-align: left; background-color: #050A0E; border: 1px solid lightblue;">
    <tr>
        <td style="direction: rtl; text-align: right; color: #FFF8E7; ">
            <h2 style="color:#FFF8E7;">JobTonic</h2>
            <span style="color:#FFF8E7; font-size: 12px">
                כי חיפוש עבודה ראוי להרגיש כמו התחלה חדשה, לא כמו מסע מייגע.
                במקום להיסחף בגלים של מידע, 
                תנו לנו להיות הרוח שמכוונת את המפרש.
                הכלי
                 מזקק עבורכם את מהות כל משרה — בבהירות, במהירות, ובדיוק שמחזיר לכם שליטה.
                זה לא רק לחפש עבודה. זה לדייק מטרה. זה להתקדם עם ראש צלול, לב פתוח, וביטחון אמיתי.
                בכל שלב בדרך – אנחנו האנרגיה שמזיזה אתכם קדימה.
            </span>
        </td>
<td style="width: 290px; height: 200px; vertical-align: middle;">
            <img src="JobTonic.png" style="width: 100%; height: 100%; display: block;" />
        </td>
    </tr>
</table>

In [106]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
import anthropic
import google.generativeai
import ollama
import json

In [94]:
# Constants
MODEL = "llama3.2"

# Load environment variables
load_dotenv(override=True)

# Initialize Google and Anthropic clients
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
MODEL_Claude = "claude-3-7-sonnet-latest"
claude = anthropic.Anthropic()

google_api_key = os.getenv('GOOGLE_API_KEY')
MODEL_Google = "gemini-2.0-flash"
google.generativeai.configure()

api_key = os.getenv('AZURE_OPENAI_API_KEY')
endpoint = os.getenv('ENDPOINT')
version = os.getenv('VERSION')
deployment = os.getenv('DEPLOYMENT_4_1nano')

client = AzureOpenAI(
    azure_endpoint=endpoint, 
    api_key=api_key,
    api_version=version
)

In [95]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with Selenium for dynamic websites.
    """

    def __init__(self, url):
        self.url = url

        # Set up Selenium WebDriver with Chrome
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')  # Run in headless mode
        options.add_argument('--disable-gpu')
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

        # Load the webpage
        driver.get(url)
        time.sleep(5)  # Wait for the page to load completely

        # Get the page source and parse it with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit()  # Close the browser

        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [143]:
# The system prompt for job postings
job_posting_system_prompt = """
    You are an assistant specialized in summarizing job postings.\n
    Your task is to extract and clearly summarize three things from the input job description:\n
    1. How the company describes itself.\n
    2. What the day-to-day responsibilities of the role are.\n
    3. What the qualifications or requirements are to apply.\n\n
    You will be provided with a link to job posting. for example:\n
    https://www.example.com/job/software-engineer or https://www.example.com/job/data-scientist you should extract the relevant information 
    from the job description and write a summary in plain, professional English. Keep each section concise (1–2 sentences).\n
    Do not copy text verbatim unless necessary. Structure your response under the following headers:\n
    - About the Company\n
    - Role Responsibilities\n
    - Qualifications and Requirements\n
    in addition to the following fields:\n
    - Company Name\n
    - Job URL\n
    - Company Location\n
    - Company Size\n
    keep headers empty if the information is not available.\n
"""
job_posting_system_prompt += "\nYou should respond in JSON as in this example:"
job_posting_system_prompt += """
{
    "company_name": "Tech Innovations Inc.",
    "job_url": "https://full.url/goes/here/software-engineer",
    "company_location": "San Francisco, CA",
    "company_size": "500+ employees",
    "company_description": "The company is a leading provider of innovative technology solutions.",
    "role_responsibilities": "The role involves developing software applications and collaborating with cross-functional teams.",
    "qualifications_requirements": "Candidates should have a degree in Computer Science and experience with Python."
}
"""
job_posting_system_prompt += "\nIn the case that there are no relevant links, respond with an empty JSON object: {}"

job_posting_system_prompt += "\n here is a real example of a job posting:\n"
job_posting_system_prompt += """
{
  "company_name": "Earnix",
  "job_url": "https://earnix.com/career/0d.f45/automation-engineer/",
  "company_location": "Ramat Gan, Israel",
  "company_size": "201–500 employees",
  "company_description": "Earnix is a premier provider of cloud-based intelligent decisioning solutions for pricing, rating, underwriting, and product personalization in the insurance and banking sectors, serving clients across over 35 countries.",
  "role_responsibilities": "Develop and enhance server-side automation tests and infrastructure, analyze test results, participate in code reviews, and collaborate with cross-functional teams to ensure high-quality product delivery.",
  "qualifications_requirements": "Minimum 3 years of experience in server-side automation development using Python/Pytest, proficiency with AWS, Docker, Jenkins, Git, and Linux environments, a bachelor's degree in Computer Science or related field, with a background in mathematics or statistics considered an advantage."
}
"""

job_posting_system_prompt += "\n here is another real example of a job posting:\n"
job_posting_system_prompt += """
{
  "company_name": "Earnix",
  "job_url": "https://earnix.com/career/0d.f45/automation-engineer/",
  "company_location": "Givatayim, Israel",
  "company_size": "201-500 employees",
  "company_description": "Earnix is a global provider of AI-driven rating, pricing, and product personalization solutions for insurance and banking, helping financial institutions transform how they make decisions with real-time, dynamic, and integrated analytics.",
  "role_responsibilities": "The Automation Engineer will develop and maintain test automation frameworks, create and execute automated test scripts, collaborate with development teams on CI/CD pipelines, and continuously improve testing methodologies to ensure high-quality software releases.",
  "qualifications_requirements": "Candidates need 3+ years of experience in test automation, proficiency in Python, knowledge of API testing frameworks like REST-assured or Postman, familiarity with CI/CD tools, and strong analytical and problem-solving skills."
}
"""

In [144]:
# This function is used to create a user prompt to find the job postings
def get_job_postings_user_prompt(text):
    user_prompt = """here is a link to a job posting. read the content inside the link and extract the job posting from it. \n
    Write the summary to that job posting.\n"""
    user_prompt += text
    return user_prompt

In [173]:
is_job_relevant_system_prompt = """ You will receive:
1. job description summaries (including role, requirements, and expectations).
2. A resume in free-text/plain-text format.

Your task is to evaluate whether the jobs appear relevant to the candidate based on the skills, experiences, and qualifications in the resume.

Instructions:
- Compare the core requirements and desired qualifications from the job description to the candidate's experience,
 skills, education, and achievements in the resume.
- Focus on key match indicators such as:
  - Required technologies, tools, or methodologies
  - Years of experience
  - Industry relevance
  - Certifications or degrees
  - Role-specific accomplishments

Respond with the following structure:
```json
{
  "relevant": true or false,
  "reasoning": "A concise explanation of why the job is or isn't relevant to the candidate."
}
IF there are more than 3 relevant jobs, return the most relevant 3 jobs only."""


In [174]:
def get_job_relevant_user_prompt(resume, job_summaries):
    user_prompt = "Here are the job description summaries and the candidate's resume. " \
    "Please assess whether the jobs are relevant to the candidate. for the jobs that are relevant, " \
    "please choose top 3 relevant jobs if you find more than 3.\n\n"
    user_prompt += """The resume: \n\n"""
    user_prompt += resume
    user_prompt += """\n\n
    The jobs descriptions: \n\n"""
    user_prompt += job_summaries
    # print(user_prompt)
    return user_prompt

In [107]:
import fitz  # PyMuPDF

def read_pdf_text(filepath):
    try:
        text = ""
        with fitz.open(filepath) as doc:
            for page in doc:
                text += page.get_text()
        return text
    except Exception as e:
        return f"Error reading PDF: {e}"
    
def read_website(url):
    try:
        website = Website(url)
        return website.text
    except Exception as e:
        return f"Error reading website: {e}"
    
def read_text_file(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            text = file.read()
        return text
    except Exception as e:
        return f"Error reading text file: {e}"

In [111]:
read_website_function = {
    "name": "read_website",
    "description": "Get the content of the website by its link. Call this whenever you need to read link content, for example when a customer provides a link to a job posting.",
    "parameters": {
        "type": "object",
        "properties": {
            "link": {
                "type": "string",
                "description": "The link to the website you want to read. For example: https://www.example.com/job/software-engineer",
            },
        },
        "required": ["link"],
        "additionalProperties": False
    }
}

In [162]:
def call_tool(reply, messages): 
    tool_responses = []
    messages.append(reply)

    for tool_call in reply.tool_calls:
        if tool_call.function.name == "read_website":
            try:
                # print(tool_call)
                arguments = json.loads(tool_call.function.arguments)
                # print(arguments)
                link = arguments.get('link')
                print(link)
                # Validate the link
                if not link or not link.startswith("http"):
                    raise ValueError(f"Invalid link: {link}")
                
                website_text = read_website(link)
                tool_responses.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": website_text
                })
                # print(messages)
            except Exception as e:
                # Handle errors gracefully
                error_message = f"Error processing tool_call_id {tool_call.id}: {e}"
                tool_responses.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": error_message
                })

        else:
            error_message = f"Error processing tool_call_id {tool_call.id}: Unsupported function {tool_call.function.name}"
            tool_responses.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": error_message
            })

    messages.extend(tool_responses)
    followup = client.chat.completions.create(
        model=deployment,
        messages=messages
    )

    # print("🔁 Model's final response:")
    # print(followup.choices[0].message.content)
    return followup.choices[0].message.content

In [163]:
# generic API call to 4 models
def callModel(company, system_prompt, user_prompt):
    print(f"Calling {company}")
    if company == "ollama":
        response = ollama.chat(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
        )
        return response['message']['content']
    elif company == "openai":
        messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        tools = [{"type": "function", "function": read_website_function}]
        response = client.chat.completions.create(
            model=deployment,
            messages=messages,
            tools=tools
        )
        reply = response.choices[0].message
        if response.choices[0].finish_reason=="tool_calls":
            final_response = call_tool(reply, messages)
            return final_response
        else:
            return reply.content
    elif company == "anthropic":
        response = claude.messages.create(
            model=MODEL_Claude,
            max_tokens=500,
            temperature=0.4,
            system=system_prompt,
            messages=[
                {"role": "user", "content": user_prompt},
            ],
        )
        return response.content[0].text
    elif company == "google":
        gemini = google.generativeai.GenerativeModel(
            model_name=MODEL_Google,
            system_instruction=system_prompt,
            )
        response = gemini.generate_content(user_prompt)
        return response.text


In [179]:
# summarize the job postings
def summarize_job_postings(job_postings):
    # Call the model to summarize the job postings
    user_prompt = get_job_postings_user_prompt(job_postings)
    response = callModel("openai", job_posting_system_prompt, user_prompt)
    return response

def is_job_relevant(resume, job_summaries):
    # Call the model to check if the job is relevant
    user_prompt = get_job_relevant_user_prompt(resume, job_summaries)
    response = callModel("anthropic", is_job_relevant_system_prompt, user_prompt)
    return response

In [180]:
text = read_text_file("job_links.txt")
links = [link.strip() for link in text.split(",") if link.strip()]

all_summaries = []
for link in links:
    try:
        summary = summarize_job_postings(link)
        all_summaries.append({
            "link": link,
            "summary": summary
        })
    except Exception as e:
        print(f"❌ Error processing {link}: {e}")
        all_summaries.append({
            "link": link,
            "summary": f"Error: {e}"
        })

Calling openai
https://octosource.io/jobs/data-scientist/


In [181]:
print("All summaries:")
for summary in all_summaries:
    print(f"Link: {summary['link']}")
    print(f"Summary: {summary['summary']}")
    print("-" * 80)

All summaries:
Link: [
"https://octosource.io/jobs/data-scientist/"]
Summary: {
    "company_name": "OctoSource Ltd",
    "job_url": "https://octosource.io/jobs/data-scientist/",
    "company_location": "Israel",
    "company_size": "",  // Not specified
    "company_description": "OctoSource is a recruitment company specializing in hiring top talent in the tech industry, with over 10 years of experience in sourcing candidates in fields such as Data Science, software development, cybersecurity, and more.",
    "role_responsibilities": "The Data Scientist will analyze and deploy models for real-time bidding systems, lead research in machine learning algorithms, and translate business challenges into data pipelines and models, with a focus on high performance and real-time feedback.",
    "qualifications_requirements": "Candidates need at least 8 years of experience in a data science environment, proficiency in Python (Scala is a plus), experience with big data and distributed computing 

In [182]:
resume = read_pdf_text("SapirShamay.pdf")
summaries = "/n".join([f"link: {summary['link']}\n summary:\n{summary['summary']}\n" for summary in all_summaries])
response = is_job_relevant(resume, summaries)
print("Job relevance response:")
print(response)

Calling anthropic


Job relevance response:
I've analyzed the resume and job description. Here's my assessment:

```json
{
  "relevant": false,
  "reasoning": "The job requires at least 8 years of experience in a data science environment and an advanced degree (MSc or Ph.D.), while the candidate has primarily software engineering experience (not specifically data science) and only a BSc degree. Although the candidate has some data analysis experience and mentions basic AI/ML tools in their skills, they lack the extensive data science background, big data experience with tools like Spark (only listed as a skill, not demonstrated in work experience), and the advanced degree required for this senior Data Scientist position."
}
```
